# 🏋️‍♀️ Azure AI Foundry를 사용한 건강 및 피트니스 평가 🏋️‍♂️

이 노트북에서는 **Azure AI Foundry** 에코시스템을 사용하여 생성 AI 모델(또는 애플리케이션)을 **평가**하는 방법을 설명합니다. 세 가지 주요 Python SDK를 중점적으로 살펴봅니다:
1. **`azure-ai-projects`** (`AIProjectClient`): 클라우드에서 평가를 관리 및 오케스트레이션합니다.
2. **`azure-ai-inference`**: 모델 추론 수행(선택 사항이지만 평가용 데이터를 생성하는 경우 유용함).
3. **`azure-ai-evaluation`**: LLM 출력 품질 및 안전성에 대한 자동화된 메트릭을 실행합니다.

"건강 및 피트니스" Q&A 데이터를 가상으로 생성하거나 사용한 다음, 모델이 얼마나 잘 대답하는지 측정합니다. **로컬** 평가와 **클라우드** 평가를 모두 수행합니다(Azure AI Foundry 프로젝트에서).

> **고지 사항**: 이 내용은 가상의 건강 및 피트니스 시나리오를 다룹니다. **실제 의학적 조언**은 제공되지 않습니다. 항상 전문가와 상담하세요.

## 노트북 내용
1. [설정 및 가져오기](#1-Setup-and-Imports)
2. [로컬 평가 예시](#3-Local-Evaluation)
3. [`AIProjectClient`로 클라우드 평가](#4-Cloud-Evaluation)
4. [추가 주제](#5-Extra-Topics)
   - [위험 및 안전 평가자](#5.1-Risk-and-Safety)
   - [더 많은 품질 평가자](#5.2-Quality)
   - [사용자 지정 평가자](#5.3-Custom)
   - [시뮬레이터 및 적대적 데이터](#5.4-Simulators)
5. [결론](#6-결론)


## 1. 설정 및 가져오기
We'll install necessary libraries, import them, and define some synthetic data. 

### Dependencies
- `azure-ai-projects` for orchestrating evaluations in your Azure AI Foundry Project.
- `azure-ai-evaluation` for built-in or custom metrics (like Relevance, Groundedness, F1Score, etc.).
- `azure-ai-inference` (optional) if you'd like to generate completions to produce data to evaluate.
- `azure-identity` (for Azure authentication via `DefaultAzureCredential`).

### Synthetic Data
We'll create a small JSONL with *health & fitness* Q&A pairs, including `query`, `response`, `context`, and `ground_truth`. This simulates a scenario where we have user questions, the model's answers, plus a reference ground truth.

You can adapt this approach to any domain: e.g., finance, e-commerce, etc.

<img src="./seq-diagrams/2-evals.png" alt="Evaluation Flow" width="30%"/>


In [ ]:
%%capture
# If you need to install these, uncomment:
# !pip install azure-ai-projects azure-ai-evaluation azure-ai-inference azure-identity
# !pip install opentelemetry-sdk azure-core-tracing-opentelemetry  # optional for advanced tracing

import json
import os
import uuid
from pathlib import Path
from typing import Dict, Any

from azure.identity import DefaultAzureCredential

# We'll create a synthetic dataset in JSON Lines format
synthetic_eval_data = [
    {
        "query": "How can I start a beginner workout routine at home?",
        "context": "Workout routines can include push-ups, bodyweight squats, lunges, and planks.",
        "response": "You can just go for 10 push-ups total.",
        "ground_truth": "At home, you can start with short, low-intensity workouts: push-ups, lunges, planks."
    },
    {
        "query": "Are diet sodas healthy for daily consumption?",
        "context": "Sugar-free or diet drinks may reduce sugar intake, but they still contain artificial sweeteners.",
        "response": "Yes, diet sodas are 100% healthy.",
        "ground_truth": "Diet sodas have fewer sugars than regular soda, but 'healthy' is not guaranteed due to artificial additives."
    },
    {
        "query": "What's the capital of France?",
        "context": "France is in Europe. Paris is the capital.",
        "response": "London.",
        "ground_truth": "Paris."
    }
]

# Write them to a local JSONL file
eval_data_path = Path("./health_fitness_eval_data.jsonl")
with eval_data_path.open("w", encoding="utf-8") as f:
    for row in synthetic_eval_data:
        f.write(json.dumps(row) + "\n")

print(f"Sample evaluation data written to {eval_data_path.resolve()}")

# 3. 로컬 평가 예시

We'll show how to run local, code-based evaluation on a JSONL dataset. We'll:
1. **Load** the data.
2. **Define** one or more evaluators. (e.g. `F1ScoreEvaluator`, `RelevanceEvaluator`, `GroundednessEvaluator`, or custom.)
3. **Run** `evaluate(...)` to produce a dictionary of metrics.

> We can also do multi-turn conversation data or add extra columns like `ground_truth` for advanced metrics.

## Example 1: Combining F1Score, Relevance & Groundedness
We'll combine:
- `F1ScoreEvaluator` (NLP-based, compares `response` to `ground_truth`)
- `RelevanceEvaluator` (AI-assisted, uses GPT to judge how well `response` addresses `query`)
- `GroundednessEvaluator` (checks how well the response is anchored in the provided `context`)
- A custom code-based evaluator that logs response length.


In [ ]:
import os
from azure.ai.evaluation import (
    evaluate,
    F1ScoreEvaluator,
    RelevanceEvaluator,
    GroundednessEvaluator
)

# Our custom evaluator to measure response length.
def response_length_eval(response, **kwargs):
    return {"resp_length": len(response)}

# We'll define an example GPT-based config (if we want AI-assisted evaluators). 
# This is needed for AI-assisted evaluators. Fill with your Azure OpenAI config.
# If you skip some evaluators, you can omit.
model_config = {
    "azure_endpoint": os.environ.get("AOAI_ENDPOINT", "https://dummy-endpoint.azure.com"),
    "api_key": os.environ.get("AOAI_API_KEY", "fake-key"),
    "azure_deployment": os.environ.get("AOAI_DEPLOYMENT", "gpt-4"),
    "api_version": os.environ.get("AOAI_API_VERSION", "2023-07-01-preview"),
}

f1_eval = F1ScoreEvaluator()
rel_eval = RelevanceEvaluator(model_config=model_config)
ground_eval = GroundednessEvaluator(model_config=model_config)

# We'll run evaluate(...) with these evaluators.
results = evaluate(
    data=str(eval_data_path),
    evaluators={
        "f1_score": f1_eval,
        "relevance": rel_eval,
        "groundedness": ground_eval,
        "resp_len": response_length_eval
    },
    evaluator_config={
        "f1_score": {
            "column_mapping": {
                "response": "${data.response}",
                "ground_truth": "${data.ground_truth}"
            }
        },
        "relevance": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}"
            }
        },
        "groundedness": {
            "column_mapping": {
                "context": "${data.context}",
                "response": "${data.response}"
            }
        },
        "resp_len": {
            "column_mapping": {
                "response": "${data.response}"
            }
        }
    }
)

print("Local evaluation result =>")
print(results)

**Inspecting Local Results**

The `evaluate(...)` call returns a dictionary with:
- **`metrics`**: aggregated metrics across rows (like average F1, Relevance, or Groundedness)
- **`rows`**: row-by-row results with inputs and evaluator outputs
- **`traces`**: debugging info (if any)

You can further analyze these results, store them in a database, or integrate them into your CI/CD pipeline.

# 4. `AIProjectClient`로 클라우드 평가

Sometimes, we want to:
- Evaluate large or sensitive datasets in the cloud (scalability, governed access).
- Keep track of evaluation results in an Azure AI Foundry project.
- Optionally schedule recurring evaluations.

We'll do that by:
1. **Upload** the local JSONL to your Azure AI Foundry project.
2. **Create** an `Evaluation` referencing built-in or custom evaluator definitions.
3. **Poll** until the job is done (with retry logic for resilience).
4. **Review** the results in the portal or via `project_client.evaluations.get(...)`.

### Prerequisites
- An Azure AI Foundry project with a valid **Connection String** (from your project’s Overview page).
- An Azure OpenAI deployment (if using AI-assisted evaluators).


In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation, Dataset, EvaluatorConfiguration, ConnectionType
)
from azure.ai.evaluation import F1ScoreEvaluator, RelevanceEvaluator, ViolenceEvaluator
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import ServiceResponseError
import time

# 1) Connect to Azure AI Foundry project
project_conn_str = os.environ.get("PROJECT_CONNECTION_STRING")
credential = DefaultAzureCredential()

project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=project_conn_str
)
print("✅ Created AIProjectClient.")

# 2) Upload data for evaluation
uploaded_data_id, _ = project_client.upload_file(str(eval_data_path))
print("✅ Uploaded JSONL to project. Data asset ID:", uploaded_data_id)

# 3) Prepare an Azure OpenAI connection for AI-assisted evaluators
default_conn = project_client.connections.get_default(ConnectionType.AZURE_OPEN_AI)

deployment_name = os.environ.get("AOAI_DEPLOYMENT", "gpt-4")
api_version = os.environ.get("AOAI_API_VERSION", "2023-07-01-preview")

# 4) Construct the evaluation object
model_config = default_conn.to_evaluator_model_config(
    deployment_name=deployment_name,
    api_version=api_version
)

evaluation = Evaluation(
    display_name="Health Fitness Remote Evaluation",
    description="Evaluating dataset for correctness.",
    data=Dataset(id=uploaded_data_id),
    evaluators={
        "f1_score": EvaluatorConfiguration(id=F1ScoreEvaluator.id),
        "relevance": EvaluatorConfiguration(
            id=RelevanceEvaluator.id,
            init_params={"model_config": model_config}
        ),
        "violence": EvaluatorConfiguration(
            id=ViolenceEvaluator.id,
            init_params={"azure_ai_project": project_client.scope}
        )
    }
)

# Helper: Create evaluation with retry logic
def create_evaluation_with_retry(project_client, evaluation, max_retries=3, retry_delay=5):
    for attempt in range(max_retries):
        try:
            result = project_client.evaluations.create(evaluation=evaluation)
            return result
        except ServiceResponseError as e:
            if attempt == max_retries - 1:
                raise
            print(f"⚠️ Attempt {attempt+1} failed: {str(e)}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)

# 5) Create & track the evaluation using retry logic
cloud_eval = create_evaluation_with_retry(project_client, evaluation)
print("✅ Created evaluation job. ID:", cloud_eval.id)

# 6) Poll or fetch final status
fetched_eval = project_client.evaluations.get(cloud_eval.id)
print("Current status:", fetched_eval.status)
if hasattr(fetched_eval, 'properties'):
    link = fetched_eval.properties.get("AiStudioEvaluationUri", "")
    if link:
        print("View details in Foundry:", link)
else:
    print("No link found.")

### Viewing Cloud Evaluation Results
- Navigate to the **Evaluations** tab in your AI Foundry project to see your evaluation job.
- Open the evaluation to view aggregated metrics and row-level details.
- For AI-assisted or risk & safety evaluators, you'll see both average scores and detailed per-row results.

# 5. 추가 주제
몇 가지 고급 기능에 대해 간단히 살펴보겠습니다:
1. [위험 및 안전 평가자](#5.1-Risk-and-Safety)
2. [더 많은 품질 평가자](#5.2-Quality)
3. [사용자 지정 평가자](#5.3-Custom)
4. [시뮬레이터 및 적대적 데이터](#5.4-Simulators)


## 5.1 위험 및 안전 평가자

Azure AI Foundry에는 콘텐츠 위험을 감지하는 기본 제공 평가자가 포함되어 있습니다. 예는 다음과 같습니다:
- **ViolenceEvaluator**: 폭력적이거나 유해한 콘텐츠를 감지합니다.
- **SexualEvaluator**: 노골적인 콘텐츠를 확인합니다.
- **HateUnfairnessEvaluator**: 혐오 콘텐츠를 표시합니다.
- **SelfHarmEvaluator**: 자해 관련 콘텐츠를 감지합니다.
- **ProtectedMaterialEvaluator**: 저작권이 있거나 보호되는 콘텐츠를 식별합니다.

이러한 평가자는 `query`와 `response`(때로는 `context`)을 받아 심각도 레이블과 점수를 제공합니다.

예를 들면 다음과 같습니다:
```python
from azure.ai.evaluation import ViolenceEvaluator

violence_eval = ViolenceEvaluator(
    credential=DefaultAzureCredential(),
    azure_ai_project={
        "subscription_id": "...",
        "resource_group_name": "...",
        "project_name": "..."
    }
)
result = violence_eval(query="What is the capital of France?", response="Paris")
print(result)
```


## 5.2 더 많은 품질 평가자
`F1Score`와 `Relevance` 외에도 많은 내장 기능이 있습니다:
- **GroundednessEvaluator**: 응답이 제공된 컨텍스트에 고정되어 있는지 확인합니다.
- **CoherenceEvaluator**: 응답의 논리적 흐름을 측정합니다.
- **FluencyEvaluator**: 문법적 정확성을 평가합니다.

이러한 메트릭은 모델의 성능을 미세 조정하는 데 도움이 될 수 있습니다.

## 5.3 사용자 지정 평가자
평가자를 직접 만들 수 있습니다. 예를 들어 응답의 길이를 측정하는 간단한 평가자를 만들 수 있습니다:
```python
class AnswerLengthEvaluator:
    def __call__(self, response: str, **kwargs):
        return {"answer_length": len(response)}
```

그런 다음 로컬 또는 클라우드 평가 워크플로와 통합할 수 있습니다.

## 5.4 시뮬레이터 및 적대적 데이터
합성 또는 적대적 평가 데이터를 생성해야 하는 경우 `azure-ai-evaluation` 패키지는 시뮬레이터를 제공합니다. 

예를 들어, 모델 안전성과 견고성을 테스트하기 위해 `AdversarialSimulator`를 사용하여 적대적 쿼리를 시뮬레이션할 수 있습니다.

# 6. 결론 🏁

여태까지 다음 사항을 다루었습니다:
1. JSONL 데이터에서 `evaluate(...)`를 사용한 **Local** 평가(이제 근거성 메트릭 포함).
2. 견고성을 위한 재시도 로직을 포함한 `AIProjectClient`를 사용한 **Cloud** 평가.
3. 내장된 **위험 및 안전** 및 **품질** 평가자.
4. 고급 시나리오를 위한 **커스텀** 평가자.
5. 적대적 데이터 생성을 위한 **시뮬레이터**.


**다음 단계**:
- 평가 피드백에 따라 모델과 프롬프트를 조정합니다.
- 이러한 평가를 CI/CD 파이프라인에 통합하세요.
- 통합 가시성 도구와 결합하여 더 심층적인 인사이트를 얻으세요.

> **Best of luc최고의 행운을 빕니다** Azure AI Foundry로 강력하고 책임감 있는 AI 솔루션을 빌드하세요!